### UDA Assignment 4: REI Instagram Engagement Analysis

Created by: Aman Bhardwaj, Blake DeLong, Apurva Harsulkar, Colby Meline, Joel Nail, and Rahul Rangarao

#### Introduction

Rather than analyze National Geographic's Instagram page, we chose to investigate REI, also known as Recreational Equipment, Inc. REI mainly sells camping and hiking gear, and they are known for being an "outdoorsy" brand that encourages its customers to be physically active and enjoy the beauty of nature. Compared to National Geographic's Instagram page, REI's account features more posts related to hiking, camping, and other outdoor activities. While REI features fewer animals that Nat Geo, they share a common appreciation for nature and the beautiful landscapes it has to offer. We believe that by analyzing REI's posts, we can offer valuable insights that will help the brand increase its Instagram engagement.

#### Scraper

We initially created a scraper using Selenium to extract posts from REI's instagram page. We later discovered a service called PhantomBuster which allowed us to easily scrape everything we needed from REI's account. We have included the Selenium code below as reference, but the data we analyzed was obtained from PhantomBuster

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException


from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By


# Setup Firefox
service = Service(executable_path="/home/colby/selenium/geckodriver")
firefox_options = Options()
firefox_options.binary = FirefoxBinary("/bin/firefox")
firefox_options.headless = False
driver = webdriver.Firefox(options=firefox_options, service=service)
a = ActionChains(driver)


insta_page = "https://www.instagram.com/rei/"
driver.get(insta_page)


def driver_login():
    username = driver.find_element(by=By.NAME, value="username")
    password = driver.find_element(by=By.NAME, value="password")
    submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

    username.clear()
    password.clear()
    username.send_keys("msitm_student")
    time.sleep(2)
    password.send_keys("UDA4Project")
    submit_button.click()
    time.sleep(10)

    try:
        save_info_not_now = driver.find_element(by=By.XPATH, value="//button[contains(text(), 'Not Now')]").click()
        time.sleep(10)
    except NoSuchElementException:
        print("No save info redirect")
        
    try:
        notif_not_now = driver.find_element(by=By.XPATH, value="//button[contains(text(), 'Not Now')]").click()
        time.sleep(10)
    except NoSuchElementException:
        print("No notify redirect")
        
    driver.get("https://www.instagram.com/rei")
    
time.sleep(4)
driver_login()

time.sleep(8)

In [ ]:
images = driver.find_elements(by=By.TAG_NAME, value='img')
boxes = driver.find_elements(by=By.TAG_NAME, value='a')

df = {"image": [], "likes": [], "comments": []}

if False:
    for i, image, box in enumerate(zip(images, boxes)):
        print(".", sep="")
        time.sleep(2)
        images[1].location_once_scrolled_into_view
        image.location_once_scrolled_into_view
        with open("rei/rei{}.png".format(i), "wb") as file:
            file.write(images[i].screenshot_as_png)    
        a.move_to_element(image).perform()
        time.sleep(1)
    #     parent = images[i]
    #     while parent.tag_name != "a":
    #         parent = parent.find_element(By.XPATH, "..")
    #     spans = parent.find_elements(By.TAG_NAME, "span")

        spans = box.find_elements(By.TAG_NAME, "span")
        if len(spans) == 4:
            df["image"].append(image.get_attribute("src"))
            likes, comments = [i.text for i in spans if i.text]
            df["likes"].append(likes)
            df["comments"].append(comments)
    

boxes = [i for i in boxes if "/p/" in i.get_attribute("href")]
previous = set()

In [ ]:
def iterate():
    global boxes
    global previous
    for box in boxes:
        if box not in previous:
            time.sleep(1)
            previous.add(box)
            box.location_once_scrolled_into_view
            try:
                image = box.find_element(By.TAG_NAME, "img")
                with open("rei/rei{}.png".format(box.id), "wb") as file:
                    file.write(image.screenshot_as_png)  
                a.move_to_element(image).perform()
                spans = box.find_elements(By.TAG_NAME, "span")
                if len(spans) == 4:
                    df["image"].append(image.get_attribute("src"))
                    likes, comments = [i.text for i in spans if i.text]
                    df["likes"].append(likes)
                    df["comments"].append(comments)
            except NoSuchElementException:
                pass
    return previous
            
while len(df["image"]) < 300:
    iterate()
    footer = driver.find_element(by=By.TAG_NAME, value="footer")
    footer.location_once_scrolled_into_view
    time.sleep(10)
    boxes = driver.find_elements(by=By.TAG_NAME, value='a')

In [ ]:
import pandas as pd
dataframe = pd.DataFrame(df)
dataframe.to_csv("sample_rei.csv")

#### Google Vision

In order to determine what objects are present in the scraped images, we utilized Google Cloud's Vision API. The code below was used to obtain labels for each REI post

In [ ]:
import pandas as pd
import os
from google.cloud import vision

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=""  ###The Google Cloud vision creds
df=pd.read_csv("result.csv") ##Scraped from Instagram
df=df[df['type']=='Photo'].reset_index().drop('index',axis=1)[['postUrl','description','imgUrl','likeCount','commentCount']]

In [ ]:
def detect_labels_uri(uri):
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri
    response = client.label_detection(image=image)
    labels = response.label_annotations
    res=''
    for i in labels:
        res=res+' '+i.description
    return res.lstrip()

In [ ]:
df['image_text']=df['imgUrl'].apply(lambda x:detect_labels_uri(x))
grouped_df=df.groupby(['postUrl'])['image_text'].apply(lambda x: ' '.join(x)).reset_index()
grouped_df_1=df.groupby(['postUrl']).agg({'description':'max','likeCount':'max','commentCount':'max'}).reset_index()
final_df=pd.merge(grouped_df,grouped_df_1,left_on='postUrl',right_on='postUrl')

#### Engagement Metric

In order to properly evaluate the engagement of REI's Instagram posts, we first normalized the number of likes and comments on each post by dividing by the maximum number of likes and comments respectively. We then used a weighted average of these normalized scores to develop an overall engagement metric for each post. To allow for binary logistic regression, we labelled posts with an engagement metric higher than the median score as High Engagement (1) and posts with an engagement metric lower than the median score as Low Engagement (0)

In [ ]:
rei_lda_df = pd.read_csv("REI_LDA_topic.csv")
rei_lda_df.drop("Unnamed: 0", axis=1, inplace=True)

rei_norm_df = rei_lda_df.copy()

commentCountMax = max(rei_norm_df['commentCount'])
likeCountMax = max(rei_norm_df['likeCount'])

rei_norm_df['commentCountNorm'] = rei_norm_df['commentCount'] / commentCountMax
rei_norm_df['likeCountNorm'] = rei_norm_df['likeCount'] / likeCountMax

# post 13 has the highest comment count
# post 38 has the highest like count

# since likes are cheap and comments can be hard to come by, we have weighted our comment count higher than our like count
# this assumes that REI prioritizes comments over likes when it comes to engagement
rei_norm_df['engagementMetric'] = (rei_norm_df['commentCountNorm'] * .6 + rei_norm_df['likeCountNorm'] * .4)

#print(rei_norm_df['engagementMetric'].idxmax())

median_engagement_score = np.median(rei_norm_df['engagementMetric'])
print(median_engagement_score)

rei_norm_df['engagementBinary'] = np.where(rei_norm_df['engagementMetric'] < median_engagement_score, 0, 1)

#### Logistic Regression

We utilized tf-idf scores in order to perform word replacement on our image labels and post captions. Following this step, we performed three different logistic regressions to predict engagement. The first regression used only image labels, the second regression utilized the post captions, and the final regression used both image labels and post captions

In [ ]:
import numpy
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('REI_LDA_topic.csv').drop([185]).reset_index()

string_punctuation = '''()-[]{};:'"\,<>./?@#$%^&*_~1234567890'''
stop = stopwords.words('english')

def remove_punctuation(s):
    no_punct = ""
    for letter in s:
        if letter not in string_punctuation:
            no_punct += letter
    return no_punct

stemmer = SnowballStemmer("english")

TF-IDF for Captions

In [ ]:
df.iloc[ :, 3] = df.iloc[ :, 3].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

i = 0

for row in df['description']:
    df.iloc[ i, 3] = remove_punctuation(row)
    i=i+1
df['description'] = df['description'].str.replace("!"," !")
df['description'] = df['description'].apply(word_tokenize)
df['description'] = df['description'].apply(lambda x: [stemmer.stem(y) for y in x])
df['description'] = df['description'].apply(lambda x : " ".join(x))

Text_Column = df.iloc[ :, 3]

sklearn_tfidf = TfidfVectorizer(min_df=.01, max_df =.95, stop_words="english",use_idf=True, smooth_idf=False, sublinear_tf=True)
sklearn_representation = sklearn_tfidf.fit_transform(Text_Column.tolist())
Tfidf_Output_caption = pd.DataFrame(sklearn_representation.toarray(), columns=sklearn_tfidf.get_feature_names_out())

TF-IDF for Image Descriptions

In [ ]:
df = pd.read_csv('REI_LDA_topic.csv').drop([185]).reset_index()

string_punctuation = '''()-[]{};:'"\,<>./?@#$%^&*_~1234567890'''
stop = stopwords.words('english')

In [ ]:
df.iloc[ :, 2] = df.iloc[ :, 2].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

i = 0

for row in df['image_text']:
    df.iloc[ i, 2] = remove_punctuation(row)
    i=i+1
df['image_text'] = df['image_text'].str.replace("!"," !")
df['image_text'] = df['image_text'].apply(word_tokenize)
df['image_text'] = df['image_text'].apply(lambda x: [stemmer.stem(y) for y in x])
df['image_text'] = df['image_text'].apply(lambda x : " ".join(x))

Text_Column = df.iloc[ :, 2]

In [ ]:
sklearn_tfidf = TfidfVectorizer(min_df=.01, max_df =.95, stop_words="english",use_idf=True, smooth_idf=False, sublinear_tf=True)
sklearn_representation = sklearn_tfidf.fit_transform(Text_Column.tolist())
Tfidf_Output_image = pd.DataFrame(sklearn_representation.toarray(), columns=sklearn_tfidf.get_feature_names_out())

Code to Display Confusion Matrix

In [ ]:
def show_confusion_matrix(C,class_labels=['0','1']):

    assert C.shape == (2,2), "Confusion matrix should be from binary classification only."

    # true negative, false positive, etc...
    tn = C[0,0]; fp = C[0,1]; fn = C[1,0]; tp = C[1,1];

    NP = fn+tp # Num positive examples
    NN = tn+fp # Num negative examples
    N  = NP+NN

    fig = plt.figure(figsize=(8,8))
    ax  = fig.add_subplot(111)
    ax.imshow(C, interpolation='nearest', cmap=plt.cm.gray)

    # Draw the grid boxes
    ax.set_xlim(-0.5,2.5)
    ax.set_ylim(2.5,-0.5)
    ax.plot([-0.5,2.5],[0.5,0.5], '-k', lw=2)
    ax.plot([-0.5,2.5],[1.5,1.5], '-k', lw=2)
    ax.plot([0.5,0.5],[-0.5,2.5], '-k', lw=2)
    ax.plot([1.5,1.5],[-0.5,2.5], '-k', lw=2)

    # Set xlabels
    ax.set_xlabel('Predicted Label', fontsize=16)
    ax.set_xticks([0,1,2])
    ax.set_xticklabels(class_labels + [''])
    ax.xaxis.set_label_position('top')
    ax.xaxis.tick_top()
    # These coordinate might require some tinkering. Ditto for y, below.
    ax.xaxis.set_label_coords(0.34,1.06)

    # Set ylabels
    ax.set_ylabel('True Label', fontsize=16, rotation=90)
    ax.set_yticklabels(class_labels + [''],rotation=90)
    ax.set_yticks([0,1,2])
    ax.yaxis.set_label_coords(-0.09,0.65)


    # Fill in initial metrics: tp, tn, etc...
    ax.text(0,0,
            '%d'%(tn),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(0,1,
            '%d'%fn,
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(1,0,
            '%d'%fp,
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))


    ax.text(1,1,
            '%d'%(tp),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    # Fill in secondary metrics: accuracy, true pos rate, etc...
    ax.text(2,0,
            'Error: %.2f'%(fp / (fp+tn+0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(2,1,
            'Error: %.2f'%(fn / (tp+fn+0.)),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(2,2,
            'Accuracy: %.2f'%((tp+tn+0.)/N),
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(0,2,' ',
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))

    ax.text(1,2,
            ' ',
            va='center',
            ha='center',
            bbox=dict(fc='w',boxstyle='round,pad=1'))


    plt.tight_layout()

In [ ]:
rei_df = pd.read_csv("REI_LDA_topic.csv").drop([185]).reset_index()
rei_norm_df = pd.read_csv("REI_LDA_topic.csv").drop([185]).reset_index()

commentCountMax = max(rei_df['commentCount'])
likeCountMax = max(rei_df['likeCount'])

rei_norm_df['commentCountNorm'] = rei_norm_df['commentCount'] / commentCountMax
rei_norm_df['likeCountNorm'] = rei_norm_df['likeCount'] / likeCountMax

# post 13 has the highest comment count
# post 38 has the highest like count

# since likes are cheap and comments can be hard to come by, we have weighted our comment count higher than our like count
# this assumes that REI prioritizes comments over likes when it comes to engagement
rei_norm_df['engagementMetric'] = (rei_norm_df['commentCountNorm'] * .6 + rei_norm_df['likeCountNorm'] * .4)

#print(rei_norm_df['engagementMetric'].idxmax())

median_engagement_score = np.median(rei_norm_df['engagementMetric'])
print(median_engagement_score)

rei_norm_df['engagementBinary'] = np.where(rei_norm_df['engagementMetric'] < median_engagement_score, 0, 1)

Logistic Regression with Post Captions

In [ ]:
Input = pd.concat([rei_norm_df, Tfidf_Output_caption], axis=1)

y = Input['engagementBinary'].to_list()
X = Input.loc[:, Tfidf_Output_caption.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42, test_size=0.3)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
y_pred = y_pred.tolist()

matrix = confusion_matrix(y_test, y_pred)
show_confusion_matrix(matrix, ['0', '1'])
plt.show()

![image](caption_conf.png)

Logistic Regression with Image Labels

In [ ]:
Input = pd.concat([rei_norm_df, Tfidf_Output_image], axis=1)

y = Input['engagementBinary'].to_list()
X = Input.loc[:, Tfidf_Output_image.columns]

X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42, test_size=0.3)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
y_pred = y_pred.tolist()

matrix = confusion_matrix(y_test, y_pred)
show_confusion_matrix(matrix, ['0', '1'])
plt.show()

![image](image_label_conf.png)

Logistic Regression with Image Labels and Post Captions Combined

In [ ]:
Input = pd.concat([rei_norm_df, Tfidf_Output_image, Tfidf_Output_caption], axis=1)

columns = Tfidf_Output_image.columns
columns = columns.append(Tfidf_Output_caption.columns)

y = Input['engagementBinary'].to_list()
X = Input.loc[:, columns]

X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42, test_size=0.3)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
y_pred = y_pred.tolist()

matrix = confusion_matrix(y_test, y_pred)
show_confusion_matrix(matrix, ['0', '1'])
plt.show()

![image](combined_conf.png)

As can be seen above, using post captions was the best method for predicting the engagement of REI's Instagram posts although all three method produced fairly similar results. We believe this is because REI has informative post captions that are indicative of the post's content - something that is not true for all Instagram accounts. It is important to remember that these findings are specific to REI's account and may not apply to other brands' Instagram pages

#### Topic Modeling

We performed topic modeling on both the image labels and the post captions we obtained. We began with five topics but we found too much overlap between the topics at this level. After reducing the number of topics to four, we were happy with the categories that the algorithm produced

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import lda
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))

def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(final_df['image_text'])
model = lda.LDA(n_topics=int(4), n_iter=500, random_state=1,eta=0.05)
model.fit(total_features_words)
topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
final_df=final_df.join(doc_topic)
res=pd.DataFrame()

for i in range(int(4)):
    topic="topic_"+str(i)
    res[topic]=final_df.groupby(['postUrl'])[i].mean()
res=res.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()


topics1.to_csv("topics_img.csv")

final_df=final_df.rename({0:'img_text_promos',1:'img_landscape',2:'img_campers/caravans',3:'img_outdoor_acitivies'},axis=1)

Topic Modeling for Post Captions

In [ ]:
total_features_words = vec_words.fit_transform(final_df['description'])
model = lda.LDA(n_topics=int(4), n_iter=500, random_state=1,eta=0.05)
model.fit(total_features_words)
topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
final_df=final_df.join(doc_topic)
res=pd.DataFrame()

for i in range(int(4)):
    topic="topic_"+str(i)
    res[topic]=final_df.groupby(['postUrl'])[i].mean()
res=res.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()


topics1.to_csv("topics_caption.csv")

final_df=final_df.rename({0:'cap_rei_challenges',1:'cap_outdoor_running',2:'cap_hiking_gear',3:'cap_rei_promos'},axis=1)

final_df=final_df.rename({'landscape':'img_landscape','campers/caravans':'img_campers_caravans'
            ,'outdoor_activities':'img_outdoor_acitivites'},axis=1)

final_df.to_csv("REI_LDA_topic.csv")

#### Quartile Analysis

Utilizing the topics generated above, we split our dataset into quarters based on the previously created engagement metric. We then compared the top quartile's average topic weights with the bottom quartile's average topic weights in order to determine if certain topics are more likely to encourage high engagement. You can see our findings below

In [ ]:
just_engagement = rei_norm_df['engagementMetric']

engagement_top_quartile = np.quantile(just_engagement, .75)
engagement_bottom_quartile = np.quantile(just_engagement, .25)


# code to get the index values for the top quartile
top_quart_idx = np.where(rei_norm_df['engagementMetric'] >= engagement_top_quartile, rei_norm_df.index, None)
top_quart_idx_list = np.ndarray.tolist(top_quart_idx)
for row in top_quart_idx:
    try:
        top_quart_idx_list.remove(None)
    except ValueError:
        pass
# print(len(top_quart_idx_list)) # should be 92 (1/4 of our dataset)


# code to get the index values for the bottom quartile
bottom_quart_idx = np.where(rei_norm_df['engagementMetric'] <= engagement_bottom_quartile, rei_norm_df.index, None)
bottom_quart_idx_list = np.ndarray.tolist(bottom_quart_idx)
for row in bottom_quart_idx:
    try:
        bottom_quart_idx_list.remove(None)
    except ValueError:
        pass
# print(len(bottom_quart_idx_list)) # should be 92 (1/4 of our dataset)


# now let's get dataframes for the bottom and top quartiles
top_quart_df = rei_norm_df[rei_norm_df.index.isin(top_quart_idx_list)]
bottom_quart_df = rei_norm_df[rei_norm_df.index.isin(bottom_quart_idx_list)]

In [ ]:
cols_to_analyze = ['img_text_promos', 'img_landscape', 'img_campers_caravans', 'outdoor_acitivies', 'cap_rei_challenges',
 'cap_outdoor_running', 'cap_hiking_gear', 'cap_rei_promos']

print("Top Quartile")
for col in cols_to_analyze:
    print(col + ":  \t" + str(np.mean(top_quart_df[col])))

print("\nBottom Quartile")
for col in cols_to_analyze:
    print(col + ":  \t" + str(np.mean(bottom_quart_df[col])))

In [ ]:
cols_to_analyze = ['img_text_promos', 'img_landscape', 'img_campers_caravans', 'outdoor_acitivies', 'cap_rei_challenges',
 'cap_outdoor_running', 'cap_hiking_gear', 'cap_rei_promos']

quartile_df = pd.DataFrame(columns=["Top_Quartile", "Bottom_Quartile", "Difference"], index=cols_to_analyze)

for col in cols_to_analyze:
    quartile_df.loc[col].loc['Top_Quartile'] = np.mean(top_quart_df[col])

for col in cols_to_analyze:
    quartile_df.loc[col].loc['Bottom_Quartile'] = np.mean(bottom_quart_df[col])

quartile_df["Difference"] = quartile_df["Top_Quartile"] - quartile_df["Bottom_Quartile"]

# negative Difference value indicates that it is bad for engagement
# positive Difference value indicates that it is good for engagement
quartile_df

![image](quartile_table.JPG)

#### Analysis and Advice

For REI, based on our quartile difference analysis we have found that engagement on posts is increased when the picture
is of a landscape. Post engagement is negatively affected when the caption talks about hiking gear and promotions and
they are much worse for engagement if REI posts them as an image. Customer engagement improves a lot when REI posts
about landscapes and challenges. This can be explained by analysing the platform of Instagram. Instagram users react more
positively to lifestyle posts than direct ads. From our findings it would be better for post engagement if REI posted
more about landscapes and challenges. Even though REI is a brand for gear and reseller for other brands, on instagram
REI should try to establish a lifestyle improvement presence. REI should position their pictures and captions to relate
to an active lifestyle. When making posts that feature promotions, REI must meld the promotion with their active lifestyle image in order to keep engagement with promotional posts high.

This analysis comes from our findings of high quartile difference between top quartile and bottom quartile of topic
analysis. We found that there was a high positive difference between top quartile and bottom quartile for landscape
picture posts and challenges in captions. We also found a strong negative difference for promotions in both pictures and
captions. We would also recommend that REI move text to the captions and take away all text from images.

Based on the logistic regression we performed we found that predicting customer engagement on captions is more accurate
than predicting customer engagement on attributes retrieved about images from the Google Vision API. We got a 65%
accuracy using attributes from the Google vision API vs a 70% accuracy via using just the captions. Combining the two we
got an accuracy of 68%. We found that the Google image description alone was not very useful in predicting engagement,
but rather it's value was found in the use of topic modeling where the correlation between image and engagement became
apparent.